In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 19:45:26.409273: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:45:26.443846: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:45:26.444673: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 19:45:27.444503: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
894


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9749                 235.591  -     17   +     17       False
background_yield_0.9749             858.909  -     30   +     30       False
frac_Mbc_bkg1_0.9749             0.00200584  -  0.034   +  0.035       False
frac_deltaE_bkg1_0.9749            0.016806  -  0.099   +   0.11       False
True
Iteration 1 - Signal Yield: 235.59113160358498 ± 16.647789642923215
Running iteration 2/300...
200
894
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.228                 225.455  -     16   +     17       False
background_yield_0.228             868.602  -     30   +     31       False
frac_Mbc_bkg1_0.228              0.0213721  -  0.035   +  0.036       False
frac_deltaE_bkg1_0.228            0.028667  -    0

## 150

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7101                 184.168  -     15   +     15       False
background_yield_0.7101             860.245  -     30   +     30       False
frac_Mbc_bkg1_0.7101             0.00569891  -  0.034   +  0.035       False
frac_deltaE_bkg1_0.7101           0.0239456  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 184.16811188673032 ± 14.938854485011868
Running iteration 2/300...
150
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5732                 175.936  -     15   +     15       False
background_yield_0.5732             868.141  -     30   +     31       False
frac_Mbc_bkg1_0.5732               0.023599  -  0.034   +  0.036       False
frac_delta

## 100

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8249                  129.63  -     13   +     13       False
background_yield_0.8249              865.67  -     31   +     29       False
frac_Mbc_bkg1_0.8249               0.015526  -  0.034   +  0.035       False
frac_deltaE_bkg1_0.8249           0.0345334  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 129.63012064712785 ± 12.931372838477792
Running iteration 2/300...
100
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9265                 130.102  -     13   +     13       False
background_yield_0.9265             864.123  -     30   +     30       False
frac_Mbc_bkg1_0.9265               0.014358  -  0.033   +  0.035       False
frac_delta

## 75

In [5]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1627                 105.248  -     11   +     12       False
background_yield_0.1627             863.854  -     30   +     30       False
frac_Mbc_bkg1_0.1627              0.0134865  -  0.033   +  0.035       False
frac_deltaE_bkg1_0.1627           0.0353636  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 105.24804052570565 ± 11.785109727419629
Running iteration 2/300...
75
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2039                 102.384  -     11   +     12       False
background_yield_0.2039             866.597  -     30   +     30       False
frac_Mbc_bkg1_0.2039              0.0207985  -  0.034   +  0.035       False
frac_deltaE_

## 50

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5945                 78.3265  -     10   +     11       False
background_yield_0.5945             865.686  -     30   +     30       False
frac_Mbc_bkg1_0.5945              0.0184749  -  0.033   +  0.035       False
frac_deltaE_bkg1_0.5945           0.0368932  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 78.32655285639251 ± 10.489615780657195
Running iteration 2/300...
50
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2667                 77.8959  -     10   +     11       False
background_yield_0.2667              866.03  -     30   +     30       False
frac_Mbc_bkg1_0.2667              0.0205792  -  0.033   +  0.035       False
frac_deltaE_b

## 25

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8691                 51.8985  -    8.6   +    9.3       False
background_yield_0.8691             867.076  -     30   +     30       False
frac_Mbc_bkg1_0.8691              0.0244649  -  0.033   +  0.035       False
frac_deltaE_bkg1_0.8691           0.0369331  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 51.89851255278466 ± 8.977367135331427
Running iteration 2/300...
25
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8975                 49.3174  -    8.4   +    9.5       False
background_yield_0.8975             869.535  -     30   +     30       False
frac_Mbc_bkg1_0.8975              0.0298005  -  0.033   +  0.034       False
frac_deltaE_bk

## 10

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7788                 32.8119  -    7.4   +    8.2       False
background_yield_0.7788             871.229  -     30   +     30       False
frac_Mbc_bkg1_0.7788              0.0343454  -  0.033   +  0.034       False
frac_deltaE_bkg1_0.7788            0.050608  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 32.81186017371842 ± 7.820384799739415
Running iteration 2/300...
10
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0556                 31.0296  -    7.4   +    8.1       False
background_yield_0.0556             872.926  -     30   +     30       False
frac_Mbc_bkg1_0.0556              0.0381791  -  0.033   +  0.035       False
frac_deltaE_bk

## 5

In [9]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9041                 26.6142  -    6.9   +    7.8       False
background_yield_0.9041             872.311  -     30   +     30       False
frac_Mbc_bkg1_0.9041              0.0370369  -  0.033   +  0.034       False
frac_deltaE_bkg1_0.9041           0.0523659  -  0.099   +    0.1       False
True
Iteration 1 - Signal Yield: 26.614162018812994 ± 7.343865805488521
Running iteration 2/300...
5
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4685                 25.1552  -    6.8   +    7.8       False
background_yield_0.4685             873.659  -     30   +     30       False
frac_Mbc_bkg1_0.4685              0.0403613  -  0.033   +  0.034       False
frac_deltaE_bkg

## 2

In [10]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2864                 23.1772  -    6.7   +    7.5       False
background_yield_0.2864             872.837  -     30   +     30       False
frac_Mbc_bkg1_0.2864              0.0382392  -  0.033   +  0.034       False
frac_deltaE_bkg1_0.2864           0.0564197  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 23.177241977193624 ± 7.067210370106539
Running iteration 2/300...
2
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7924                 22.1147  -    6.7   +    7.5       False
background_yield_0.7924               874.1  -     30   +     30       False
frac_Mbc_bkg1_0.7924               0.040821  -  0.033   +  0.035       False
frac_deltaE_bkg

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 1, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg1_decay_mumu_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg1_decay_mumu_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
894


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7187                 19.8622  -    6.5   +    7.3       False
background_yield_0.7187             874.101  -     30   +     30       False
frac_Mbc_bkg1_0.7187              0.0415221  -  0.033   +  0.034       False
frac_deltaE_bkg1_0.7187           0.0629093  -    0.1   +    0.1       False
True
Iteration 1 - Signal Yield: 19.86221818515888 ± 6.879266918992533
Running iteration 2/300...
0
894
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7819                 19.8622  -    6.5   +    7.3       False
background_yield_0.7819             874.101  -     30   +     30       False
frac_Mbc_bkg1_0.7819              0.0415221  -  0.033   +  0.034       False
frac_deltaE_bkg1_0.7819           0.0629093  -   